In [4]:
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# import img data and preprocessing

In [5]:
#分別抓取指定路徑下所有圖片位置，再貼上標籤(cat:1,dog:2,panda:3)
df1 = pd.DataFrame(glob.glob('C:/Users/User/OneDrive/dataset/cats/*'))
df1['label']=0
df2 = pd.DataFrame(glob.glob('C:/Users/User/OneDrive/dataset/dogs/*'))
df2['label']=1
df3 = pd.DataFrame(glob.glob('C:/Users/User/OneDrive/dataset/panda/*'))
df3['label']=2

In [6]:
#合併三個資料夾中所有路徑
df = pd.concat([df1,df2,df3],axis=0,ignore_index=True)

In [7]:
#將答案提出來
y=df['label'].values.astype('float')

In [8]:
#將指定路徑所有圖片用opencv讀取並resize成32x32，再彙整再一起
x = []
for i in df[0]:
    img = cv2.imread(i)
    img = cv2.resize(img,(64,64))
    x.append(img)
#將圖片資料格式指定成float可以提升訓練model效率
x = np.array(x,dtype='float')
#將data normalize
x = x/255

# 分割train/test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

# 建立model

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization

In [53]:
model_jmda = Sequential()

In [54]:
model_jmda.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu',input_shape=(64,64,3)))
model_jmda.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model_jmda.add(MaxPool2D(pool_size=(2, 2),padding='same'))
model_jmda.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model_jmda.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model_jmda.add(MaxPool2D(pool_size=(2, 2),padding='same'))
model_jmda.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model_jmda.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model_jmda.add(MaxPool2D(pool_size=(2, 2),padding='same'))
model_jmda.add(Flatten())
model_jmda.add(BatchNormalization())
model_jmda.add(Dense(512,activation='relu'))
model_jmda.add(BatchNormalization())
model_jmda.add(Dropout(0.3))
model_jmda.add(Dense(64,activation='relu'))
model_jmda.add(BatchNormalization())
model_jmda.add(Dense(3,activation='softmax'))
model_jmda.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 16, 16, 256)      

In [55]:
model_jmda.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

In [56]:
model_jmda.fit(x_train,y_train,batch_size=100,epochs=20,validation_data=(x_test,y_test))

Train on 2250 samples, validate on 750 samples
Epoch 1/20
2250/2250 [==============================] - 89s 40ms/sample - loss: 0.9899 - sparse_categorical_accuracy: 0.5684 - val_loss: 2.1365 - val_sparse_categorical_accuracy: 0.4680
Epoch 2/20
2250/2250 [==============================] - 94s 42ms/sample - loss: 0.6561 - sparse_categorical_accuracy: 0.6787 - val_loss: 10.6479 - val_sparse_categorical_accuracy: 0.3107
Epoch 3/20
2250/2250 [==============================] - 93s 41ms/sample - loss: 0.6108 - sparse_categorical_accuracy: 0.7067 - val_loss: 4.1587 - val_sparse_categorical_accuracy: 0.3800
Epoch 4/20
2250/2250 [==============================] - 87s 39ms/sample - loss: 0.5643 - sparse_categorical_accuracy: 0.7440 - val_loss: 0.8437 - val_sparse_categorical_accuracy: 0.6013
Epoch 5/20
2250/2250 [==============================] - 82s 36ms/sample - loss: 0.4988 - sparse_categorical_accuracy: 0.7804 - val_loss: 0.7723 - val_sparse_categorical_accuracy: 0.6320
Epoch 6/20
2250/2250 [

# Predict image

In [24]:
df_predict = pd.DataFrame(glob.glob('C:/Users/User/OneDrive/dataset/test/*'))

In [28]:
x_predict = []
for i in df_predict[0]:
    x_p = cv2.imread(i)
    x_p = cv2.resize(x_p,(64,64))
    x_predict.append(x_p)
x_predict = np.array(x_predict,dtype='float')
x_predict = x_predict/255

In [41]:
dict_animal = {0:'cat',1:'dog',2:'panda'}
matrix = np.round(model_jmda.predict(x_predict),2)
answer = pd.Series(model_jmda.predict_classes(x_predict))
answer.map(dict_animal)

0      dog
1      dog
2    panda
dtype: object

In [42]:
matrix

array([[0.07, 0.93, 0.  ],
       [0.1 , 0.87, 0.03],
       [0.  , 0.  , 1.  ]], dtype=float32)